In [1]:
import pickle, time, os
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn
from tqdm import tqdm

from visualdl import LogWriter
logwriter = LogWriter(logdir='./runs')
# visualdl --logdir ./runs/ --host 0.0.0.0 --port 8040

users_df = pd.read_csv('data/csv/users.csv')
items_df = pd.read_csv('data/csv/items.csv')

In [2]:
emb_scale = 256
batch_size = 2048
len_users = len(users_df)
len_items = len(items_df)

In [3]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.data = data
        pass
    def __getitem__(self, idx):
        return self.data[idx][0:2], self.data[idx][2]
    def __len__(self):
        return len(self.data)
    pass 

In [4]:
class Net(nn.Layer):
    def __init__(self):
        super(Net,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.user_fc1 = nn.Linear(emb_scale, 128)
        self.item_fc1 = nn.Linear(emb_scale, 128)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.cos = nn.CosineSimilarity()
        pass
    def forward(self, input):
        user = self.users_emb(input[:,0])
        item = self.items_emb(input[:,1])
        user = self.user_fc1(user)
        item = self.item_fc1(item)
        user = self.relu(user)
        item = self.relu(item)
        x = self.cos(user, item)
        x = self.sigmoid(x)
        return x
    pass
net = Net()
optim = paddle.optimizer.Adam(parameters=net.parameters(), learning_rate=0.005, weight_decay=paddle.regularizer.L2Decay(1e-3))
m = paddle.metric.Recall()

In [60]:
train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch_id in range(200):

    train_loss, train_acc, eval_loss, eval_acc = [], [], [], []

    file_list = os.listdir('data/net/train/')
    file = open('data/net/train/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size)
    file.close()    

    net.train()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = net(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss.append(loss.numpy())
        train_acc.append(acc.numpy())

        logwriter.add_scalar("train_loss", value=loss.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_acc", value=acc.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_recall", value=m.accumulate(), step=batch_id+epoch_id*(batch_size))
        
        if m.accumulate() > 0.3:
            break

    file_list = os.listdir('data/net/eval/')
    file = open('data/net/eval/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size,
                                         shuffle=True)
    file.close() 

    net.eval()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = net(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)

    logwriter.add_scalar("eval_loss", value=np.mean(eval_loss), step=epoch_id)
    logwriter.add_scalar("eval_acc", value=np.mean(eval_acc), step=epoch_id)
    logwriter.add_scalar("eval_recall", value=m.accumulate(), step=epoch_id)

    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print("epoch_id: {}, batch_id: {}, loss: {}, acc: {}, recall: {}".format(epoch_id, batch_id+1, loss.numpy(), acc.numpy(), m.accumulate()))

  1%|          | 7/839 [00:17<34:50,  2.51s/it]


KeyboardInterrupt: 